<a href="https://colab.research.google.com/github/brokentwigs/Colab-Analysis/blob/main/Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Data Choice: Images

In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
from selenium.webdriver.common.by import By
import math

ModuleNotFoundError: ignored

In [ ]:
import io
from datetime import datetime as dt
from PIL import Image
import time
import os
import requests

In [ ]:
!pip install ipython-autotime
%load_ext autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
time: 205 µs (started: 2022-08-08 15:25:00 +00:00)


In [ ]:
!pip install selenium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 981 kB 6.6 MB/s 
     |████████████████████████████████| 139 kB 48.4 MB/s 
     |████████████████████████████████| 358 kB 50.7 MB/s 
     |████████████████████████████████| 55 kB 3.8 MB/s 
     |████████████████████████████████| 4.1 MB 46.6 MB/s 
     |████████████████████████████████| 58 kB 4.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.11 which is incompatible.
time: 8.29 s (started: 2022-08-08 15:25:34 +00:00)


In [ ]:
!apt install chromium-chromedriver

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  chromium-browser chromium-browser-l10n chromium-codecs-ffmpeg-extra
Suggested packages:
  webaccounts-chromium-extension unity-chromium-extension
The following NEW packages will be installed:
  chromium-browser chromium-browser-l10n chromium-chromedriver
  chromium-codecs-ffmpeg-extra
0 upgraded, 4 newly installed, 0 to remove and 19 not upgraded.
Need to get 90.4 MB of archives.
After this operation, 306 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-codecs-ffmpeg-extra amd64 103.0.5060.134-0ubuntu0.18.04.1 [1,160 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-browser amd64 103.0.5060.134-0ubuntu0.

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=chrome_options)

In [ ]:
def get_images_from_google(driver, delay, max_images, url):
	def scroll_down(driver):
		driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
		time.sleep(delay)

	url = url
	driver.get(url)

	image_urls = set()
	skips = 0
	while len(image_urls) + skips < max_images:
		scroll_down(driver)
		thumbnails = driver.find_elements(By.CLASS_NAME, "Q4LuWd")

		for img in thumbnails[len(image_urls) + skips:max_images]:
			try:
				img.click()
				time.sleep(delay)
			except:
				continue

			images = driver.find_elements(By.CLASS_NAME, "n3VNCb")
			for image in images:
				if image.get_attribute('src') in image_urls:
					max_images += 1
					skips += 1
					break

				if image.get_attribute('src') and 'http' in image.get_attribute('src'):
					image_urls.add(image.get_attribute('src'))
					##print(f"Found {len(image_urls)}")

	return image_urls

In [ ]:
def download_image(down_path, url, file_name, image_type='JPEG',
                   verbose=True):
    try:
        time = dt.now()
        curr_time = time.strftime('%H:%M:%S')
        #Content of the image will be a url
        img_content = requests.get(url).content
        #Get the bytes IO of the image
        img_file = io.BytesIO(img_content)
        #Stores the file in memory and convert to image file using Pillow
        image = Image.open(img_file)
        file_pth = down_path + file_name

        with open(file_pth, 'wb') as file:
            image.save(file, image_type)

        if verbose == True:
            print(f'The image: {file_pth} downloaded successfully at {curr_time}.')
    except Exception as e:
        print(f'Unable to download image from Google Photos due to\n: {str(e)}')

In [ ]:
#google_urls = ['https://www.google.com/search?q=sunflower&tbm=isch&ved=2ahUKEwjN0v-4-rb5AhUYNxoKHW2OAnAQ2-cCegQIABAA&oq=sunflower&gs_lcp=CgNpbWcQAzIECCMQJzIICAAQgAQQsQMyCAgAEIAEELEDMgUIABCABDIFCAAQgAQyBQgAEIAEMgUIABCABDIFCAAQgAQyBQgAEIAEMgUIABCABDoECAAQHjoECAAQGDoECAAQQzoICAAQsQMQgwE6BwgAELEDEEM6CwgAEIAEELEDEIMBUPcGWMcNYLYOaABwAHgAgAH0AYgBwguSAQUwLjcuMZgBAKABAaoBC2d3cy13aXotaW1nwAEB&sclient=img&ei=1drwYo2UDZjuaO2cioAH&bih=969&biw=1920&rlz=1C1CHBF_en-GBGB924GB924']
labels = ['sunflower']
# if len(google_urls) != len(labels):
#         raise ValueError('The length of the url list does not match the labels list.')

    #player_path = 'images/nottingham_forest/'
     # Make the directory if it doesn't exist
for lbl in labels:
  if not os.path.exists(player_path + lbl):
    print(f'Making directory: {str(lbl)}')
    os.makedirs(player_path+lbl)

for lbl in labels:
  google_search = 'https://www.google.com/search?q='+lbl+'&tbm=isch&ved=2ahUKEwjN0v-4-rb5AhUYNxoKHW2OAnAQ2-cCegQIABAA&oq=sunflower&gs_lcp=CgNpbWcQAzIECCMQJzIICAAQgAQQsQMyCAgAEIAEELEDMgUIABCABDIFCAAQgAQyBQgAEIAEMgUIABCABDIFCAAQgAQyBQgAEIAEMgUIABCABDoECAAQHjoECAAQGDoECAAQQzoICAAQsQMQgwE6BwgAELEDEEM6CwgAEIAEELEDEIMBUPcGWMcNYLYOaABwAHgAgAH0AYgBwguSAQUwLjcuMZgBAKABAaoBC2d3cy13aXotaW1nwAEB&sclient=img&ei=1drwYo2UDZjuaO2cioAH&bih=969&biw=1920&rlz=1C1CHBF_en-GBGB924GB924'
  urls = get_images_from_google(driver, 0, 100, google_search)
  for i, url in enumerate(urls):
    download_image(down_path=f'pictures/{lbl}/',
                        url=url,
                        file_name=str(i+1)+ '.jpg',
                        verbose=True)
driver.quit()

Model Creation

In [ ]:
from keras.layers import Dense, Flatten
from keras.models import Model
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
